In [ ]:
!pip install accelerate optimum transformers

In [ ]:
!pip install datasets
!pip install auto-gptq

In [ ]:
!pip install -q bitsandbytes>=0.39.0


In [ ]:
import time
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM , GPTQConfig , BitsAndBytesConfig
from datasets import load_dataset
from tqdm import tqdm

from huggingface_hub import login
login(token="Your_token_from_hugging_face")

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model_id = "meta-llama/Llama-3.2-1B"

orig_model = AutoModelForCausalLM.from_pretrained(model_id).to(device)


# quantization_8_config = GPTQConfig(
#      bits=8,
#      group_size=128,
#      dataset="wikitext2",
#      desc_act=False,
# )

tokenizer = AutoTokenizer.from_pretrained(model_id)
# quant8_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_8_config, device_map='auto')

# quantization_4_config = GPTQConfig(
#      bits=4,
#      group_size=128,
#      dataset="wikitext2",
#      desc_act=False,
# )

# quant4_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_4_config, device_map='auto')

# quantization_3_config = GPTQConfig(
#      bits=3,
#      group_size=128,
#      dataset="wikitext2",
#      desc_act=False,
# )

# quant3_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_3_config, device_map='auto')


# quantization_2_config = GPTQConfig(
#      bits=2,
#      group_size=128,
#      dataset="wikitext2",
#      desc_act=False,
# )

# quant2_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_2_config, device_map='auto')

# model_int8 = AutoModelForCausalLM.from_pretrained(model_id,
#                                              device_map='auto',
#                                              load_in_8bit=True,
#                                              )

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

# bnb4_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})



In [ ]:
print(f"gptq4 size: {orig_model.get_memory_footprint():,} bytes")


In [ ]:
def generate_text(model, input_text, max_length=50):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    output = model.generate(inputs=input_ids,
                            max_length=max_length,
                            do_sample=True,
                            top_k=30,
                            pad_token_id=tokenizer.eos_token_id,
                            attention_mask=input_ids.new_ones(input_ids.shape))
    return tokenizer.decode(output[0], skip_special_tokens=True)



def calculate_perplexity(model, text):
    # Encode the text
    encodings = tokenizer(text, return_tensors='pt').to(device)

    # Define input_ids and target_ids
    input_ids = encodings.input_ids
    target_ids = input_ids.clone()

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)

    # Loss calculation
    neg_log_likelihood = outputs.loss

    # Perplexity calculation
    ppl = torch.exp(neg_log_likelihood)

    return ppl

In [ ]:

text4 = generate_text(quant4_model, "I have a dream")
print(f" model:\n{text4}")

In [ ]:
bnb4_perplexity = calculate_perplexity(bnb4_model, text_bnb4)
print(f"bnb4_perplexity (model): {bnb4_perplexity.item():.2f}")

In [ ]:
print(orig_model)